In [1]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("rcc").getOrCreate()

     |████████████████████████████████| 212.4MB 65kB/s 
     |████████████████████████████████| 204kB 17.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=9134efb33a5ffdf8a409ae9332e22c85eb8799ca4b48f8d184ebed151878e676
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data=spark.read.csv("/content/movielens_ratings.csv",inferSchema=True,header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
train_data,test_data=data.randomSplit([0.7,0.3])

In [10]:
als=ALS(maxIter=5,regParam=0.01,userCol="userId",itemCol="movieId",ratingCol="rating")

In [11]:
model=als.fit(train_data)

In [12]:
predictions=model.transform(test_data)

In [13]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  2.1633523|
|     31|   1.0|     5| 0.08960152|
|     31|   2.0|    25| -1.9981983|
|     31|   3.0|    14|  1.7603602|
|     85|   1.0|    28|  0.1608201|
|     85|   5.0|    16|  1.1255119|
|     85|   2.0|    20| 0.34226444|
|     85|   1.0|     5|  0.6844371|
|     85|   1.0|     4| -0.3492468|
|     85|   5.0|     8|  3.4190388|
|     85|   1.0|    23|  0.9774707|
|     85|   4.0|     7| -1.2413836|
|     85|   1.0|    25|-0.28009397|
|     85|   3.0|    21|  0.9408473|
|     65|   1.0|    28| -1.4599444|
|     65|   1.0|    22| 0.58789015|
|     65|   2.0|    15|-0.03338343|
|     65|   1.0|    24| 0.48402005|
|     53|   2.0|    19|  3.1872659|
|     53|   1.0|    23|  2.1254714|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [15]:
rmse=evaluator.evaluate(predictions)

In [16]:
rmse

2.0409171863983087

In [17]:
single_user=test_data.filter(test_data["userId"]==11).select(["movieId","userId"])

In [18]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      9|    11|
|     18|    11|
|     22|    11|
|     23|    11|
|     38|    11|
|     43|    11|
|     51|    11|
|     62|    11|
|     64|    11|
|     72|    11|
|     75|    11|
|     80|    11|
|     81|    11|
|     90|    11|
|     97|    11|
+-------+------+



In [21]:
recomendatons=model.transform(single_user)

In [22]:
recomendatons.orderBy("prediction",ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     18|    11|   5.236345|
|     43|    11|   4.865233|
|     64|    11|  3.7003465|
|     90|    11|  2.8879824|
|     97|    11|  2.0736947|
|     23|    11|  1.9909207|
|     72|    11|  1.9516172|
|      0|    11|   1.417559|
|     80|    11|  1.0348321|
|     81|    11|  0.7813848|
|     22|    11|  0.7208071|
|     51|    11|  0.7162586|
|     38|    11| 0.51689243|
|      9|    11|-0.40894783|
|     62|    11|-0.75355375|
|     75|    11| -3.7304165|
+-------+------+-----------+

